Notes:
- The way the indicator for the level of education is introduced needs to be improved. Current implementation will be unnecessarily slow when the model grows bigger.

In [1]:
import pickle
import numpy as np

As a first step, we need to ensure that all arguments we need to supply to the function are available.

In [2]:
# Import the final output of pyth_create_state_space, args
file_name = "args_file"
# Open the file for reading
file_object = open(file_name,'rb')  
# load the object from the file into var args
args = pickle.load(file_object)

In [3]:
# Unpack objects from agrs
states_all, states_number_period, mapping_states_index, max_states_period = args[0], args[1], args[2], args[3]

In [4]:
# Create additional variables:
num_periods = 10
num_choices = 3

In [5]:
# Set constants:
MISSING_INT = -99
MISSING_FLOAT = -99.00
mu = -0.56 # curvature parameter of the utility function; since it is negative, all utilities should be nonpositive

In [6]:
# Specify the values of the model parameters, optim_paras

gamma_0s1 = 5.406 # coef on b0, Blundell Table VIII, p. 1733
gamma_0s2 = 5.574
gamma_0s3 = 6.949
gamma_1s1 = 0.152 # coef on gamma0, Blundell Table VIII, p. 1733
gamma_1s2 = 0.229
gamma_1s3 = 0.306
g_s1      = 0.150 #coef on g(P), Blundell Table VIII, p. 1733
g_s2      = 0.096
g_s3      = 0.116
delta_s1  = 0.081 # coef on delta, Blundell Table VIII, p. 1733
delta_s2  = 0.057
delta_s3  = 0.073
theta_p   = 0.3 # rough average of coef values in Blundell Table IX, p. 1734
theta_f   = 0.1 # rough average of coef values in Blundell Table IX, p. 1734


optim_paras = [theta_p, 
               theta_f, 
               gamma_0s1, 
               gamma_0s2, 
               gamma_0s3, 
               gamma_1s1, 
               gamma_1s2, 
               gamma_1s3, 
               delta_s1, 
               delta_s2, 
               delta_s3, 
               g_s1, 
               g_s2, 
               g_s3]

As a second step, we need to define auxiliary functions.

In [7]:
def calculate_utilities(educ_lev, exp_p, exp_f, optim_paras):
    """Calculate systematic component of utility by choice, state, and period."""
    
    # Initialize container for utilities at state space point and period
    utilities = np.tile(np.nan, num_choices)
    
    # Calculate utilities for the avaibale joices N, P, F
    
    # Non-employment
    utilities[0] = 0
    
    #Part-time employment
    utilities[1] = 18*(np.exp(np.dot(educ_lev, optim_paras[0:3])) + 
                       np.dot(educ_lev, optim_paras[3:6]) *
                       (exp_p * np.dot(educ_lev, optim_paras[6:9]) + exp_f) 
                       * (1 - np.dot(educ_lev, optim_paras[9:12])) + 1)**mu/mu + optim_paras[12]
    
    # Full-time employment
    utilities[2] = 38*(np.exp(np.dot(educ_lev, optim_paras[0:3])) + 
                       np.dot(educ_lev, optim_paras[3:6]) *
                       (exp_p * np.dot(educ_lev, optim_paras[6:9]) + exp_f) 
                       * (1 - np.dot (educ_lev, optim_paras[9:12])) + 1)**mu/mu + optim_paras[13]
    
    return utilities

Let us provide some details concerning the above function.

Since we want to estimate coefficients which differ by the level of education of the agents (e.g., gamma, delta, g), it is convenient to create and indicator variable educ_lev. The indicator "low" is assigned if (in this example) the years of education completed are only 10, "middle" is assigned if 11 or 12, and "high" if the years of education are 13 or 14.

The utility calculation is based on hours worked. Respectively, each occupation choice is associated with a number ob hours worked. Following Blundell, in the systematic utilities calculation we use 0, 18 and 38 hours in case the current period choice is N, P, or F, respectively.

Then, we can start with the function:

In [8]:
# Initialize container
shape = (num_periods, max_states_period, num_choices)
period_utilities_systematic = np.tile(MISSING_FLOAT, shape)

In the object containing the target output of the function (period_utilities_systematic) we record the utility level net of shocks for every possible choice in the period, every admissible state space point in the period, and every period.

In [9]:
# Loop over all periods (skipping the last)
for period in range(num_periods - 1, -1, -1): # arguments are range(start, stop, step)
    
    # Loop over all admissible state space points in the period
    for k in range(states_number_period[period]):
        
        # Unpack state space components
        educ_years, choice_lagged, exp_p, exp_f = states_all[period, k, :]
        
        # Extract education information
        if (educ_years <= 10):
            educ_lev = [1,0,0]

        elif (educ_years > 10) and (educ_years <= 12):
            educ_lev = [0,1,0]

        else:
            educ_lev = [0,0,1]
        
        # Calculate rewards for each choice at state space point and period
        period_utilities_systematic[period, k, :] = calculate_utilities(educ_lev, exp_p, exp_f, optim_paras)

Export final output:

In [10]:
# Choose a file name
file_name = "period_utilities_systematic_file"

# Open the file for writing
with open(file_name,'wb') as my_file_obj:
    pickle.dump(args, my_file_obj)   